## Job Crafting: Measuring Item Bias

In [ ]:
import numpy as np
import pandas as pd

import json
from tqdm import tqdm

from scipy.spatial.distance import cosine as cosdist

In [ ]:
EMBEDDINGS = 'data/'
BIAS = 'bias/'
MODE = 'normal'  # change mode to either 'normal' or 'masked'

In [ ]:
# load pre-encoded vignette embeddings
vignette_embeddings = np.load(EMBEDDINGS + 'vignette_embeddings.npy')

In [ ]:
# load pre-encoded strategy embeddings
if MODE == 'normal':
    strategy_embeddings = np.load(EMBEDDINGS + 'strategy_embeddings.npy')
elif MODE == 'masked':
    strategy_embeddings = np.load(EMBEDDINGS + 'strategy_embeddings_masked.npy')

In [ ]:
strategies_per_item = [0, 621, 612, 611, 607]

In [ ]:
def WEAT_one_vs_all(item_no=1, random_seed=15):
    """calculates the WEAT statistic for bias dimension item_no-all_other_items:
       Are answers to item_no closer to vignette of item_no in the embedding space
       compared to the vignettes of all other items (averaged)?"""    
    start = np.sum(strategies_per_item[:item_no])
    stop = start + strategies_per_item[item_no]
    other_items = np.delete(strategy_embeddings, range(start, stop), axis=0)
    
    rng = np.random.RandomState(random_seed)
    random_idx = rng.randint(strategy_embeddings.shape[0]-strategies_per_item[item_no],
                             size=strategies_per_item[item_no])
    
    X = strategy_embeddings[start:stop]
    Y = other_items[random_idx]

    A = vignette_embeddings[item_no - 1]
    B = np.delete(vignette_embeddings, (item_no - 1), axis=0)
    
    s_WAB = []
    
    s_XAB = 0
    for sentence in X:
        inner_item_dist = cosdist(sentence, A)
        cross_item_dist = np.mean([cosdist(sentence, B[x]) for x in range(len(B))])
        s_wAB = inner_item_dist - cross_item_dist
        s_WAB.append(s_wAB)
        s_XAB += s_wAB
    mean_s_XAB = s_XAB / len(X)

    s_YAB = 0
    for sentence in Y:
        cross_item_dist = cosdist(sentence, A)
        inner_item_dist = np.mean([cosdist(sentence, B[x]) for x in range(len(B))])
        s_wAB = cross_item_dist - inner_item_dist
        s_WAB.append(s_wAB)
        s_YAB += s_wAB
    mean_s_YAB = s_YAB / len(Y)

    s_XYAB = s_XAB - s_YAB
    
    effect_size = (mean_s_XAB - mean_s_YAB) / np.std(s_WAB)

    return s_XYAB, effect_size

In [ ]:
# calculate WEAT statistics for all items: includes (bootstrapped bias statistic, bootstrapped effect size)
# save to dictionary/json

# bias_results = {}

# for item in range(1, 5):
      # compute statistic for 1000 times with different seeds to produce confidence interval
      # because of random drawing of 'other item's answers'-set
#     print(f'Bootstrapping for item {item}...')
#     bias_boot = []
#     es_boot = []
#     for seed in tqdm(np.arange(1000)):
#         WEAT = WEAT_one_vs_all(item_no=item, random_seed=seed)
#         bias_boot.append(WEAT[0])
#         es_boot.append(WEAT[1])
    
#     bias_results[item] = (bias_boot, es_boot)

In [ ]:
# save bias results to file
# with open(BIAS + f'bias_boot_{MODE}.json', 'w') as f:
    # f.write(json.dumps(bias_results))

In [ ]:
# reload pre-computed bias results
with open(BIAS + f'bias_boot_{MODE}.json', 'r') as f:
    bias_results = json.loads(f.read())

In [ ]:
# display bias results in table
df_one_vs_all = pd.DataFrame(columns=['statistic (mean)', 'CI (95%)'], index= range(1, 5))

for item in range(1, 5):
    boot = bias_results[str(item)][0]  # list of bootstrapped statistics
    boot.sort()
    
    df_one_vs_all.loc[item]['statistic (mean)'] = f'{np.mean(boot):.2f}'  # mean of bootstrapped values
    df_one_vs_all.loc[item]['CI (95%)'] = f'[{boot[int(len(boot)*0.025-1)]:.2f}; {boot[int(len(boot)*0.975-1)]:.2f}]'  # 0.025th percentile & 0.975th percentile through 25th & 750th value in sorted statistics

df_one_vs_all

In [ ]:
df_one_vs_all = pd.DataFrame(columns=['effect size (mean)', 'CI (95%)'], index= range(1, 5))

for item in range(1, 5):
    boot = bias_results[str(item)][1]  # list of bootstrapped effect sizes
    boot.sort()
    
    df_one_vs_all.loc[item]['effect size (mean)'] = f'{np.mean(boot):.2f}'  # mean of bootstrapped values
    df_one_vs_all.loc[item]['CI (95%)'] = f'[{boot[int(len(boot)*0.025-1)]:.2f}; {boot[int(len(boot)*0.975-1)]:.2f}]'  # 0.025th percentile & 0.975th percentile through 25th & 750th value in sorted effect sizes

df_one_vs_all